In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import json
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [2]:
tweet_path = "data/tweet/raw"
min_followers = 1000000

dfs = []

for symbol in tqdm(os.listdir(tweet_path)):
    for day in os.listdir(os.path.join(tweet_path, symbol)):
        file_path = os.path.join(tweet_path, symbol, day)

        with open(file_path) as f:
            content = f.readlines()
            d = list(map(json.loads, content))
            df = pd.DataFrame(d)
            df = df[df["user"].apply(lambda u: u["followers_count"]) > min_followers]
            if len(df) > 0:
                dfs.append(df)

tweets = pd.concat(dfs)
print(f"Found {len(tweets)} tweets")

simple_tweets = pd.DataFrame()
simple_tweets["date"] = pd.to_datetime(tweets["created_at"])
simple_tweets["text"] = tweets["text"].apply(lambda t: t.replace("\n", " "))
simple_tweets["user_name"] = tweets["user"].apply(lambda u: u["name"])
simple_tweets["user_followers"] = tweets["user"].apply(lambda u: u["followers_count"])
simple_tweets["sym"] = tweets["entities"].apply(lambda entities: ",".join(list(map(lambda s: s["text"], entities["symbols"]))))

simple_tweets


Found 94 tweets


,date,text,user_name,user_followers,sym
12,2016-01-28 22:51:31+00:00,Microsoft is treading water when it comes to h...,Business Insider,1446421,MSFT
2,2015-09-16 13:26:02+00:00,Why Latin American students want to work at @M...,CNNMoney,1114936,"MSFT,GOOGL"
7,2015-07-20 13:14:35+00:00,RT @StockHighAlert: #PENNYSTOCKS $PUGE $TWTR $...,BIG MONEY MIKE,2049504,"PUGE,TWTR,FB,MSFT,PUGE"
34,2016-01-28 21:11:04+00:00,RT @LaMonicaBuzz: Amazon SubPrime? Yikes. Amaz...,CNNMoney,1180864,AMZN
3,2014-08-20 15:20:05+00:00,Amazon versus the world: a recent history http...,Fortune Magazine,1326263,AMZN
...,...,...,...,...,...
2,2015-02-15 05:36:11+00:00,"RT @Cytegic: #Visa, #MasterCard to roll out ne...",Treadstone 71,1093575,"V,MA"
1,2014-12-09 01:32:04+00:00,"Americans aren't loving McDonald's food, so wh...",MarketWatch,1182937,MCD
15,2015-01-29 20:40:00+00:00,RT @jimcramer: Don't you love a stock that goe...,DR JAMES CABOT,1614099,"MCD,TWTR"
4,2015-03-04 22:02:06+00:00,McDonald's U.S. restaurants will stop selling ...,Wall Street Journal,5937170,MCD


In [3]:
# https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [28]:
tweet_in = list(simple_tweets["text"])

In [29]:
inputs = tokenizer(tweet_in, return_tensors="pt", padding=True)
outputs = model(**inputs)

In [32]:
sentiment = torch.argmax(outputs.logits, dim=1) + 1

In [33]:
simple_tweets["sentiment"] = sentiment

In [35]:
simple_tweets.to_csv("with_sent.csv")